# Fed-BioMed SecAgg Joye-Libert

Use for developing (autoreloads changes made across packages)

In [1]:
%load_ext autoreload
%autoreload 2

## Start the network
Before running this notebook, start the network with `./scripts/fedbiomed_run network`

## Setting the node up
It is necessary to previously configure a node:
1. `./scripts/fedbiomed_run node add`
  * Select option 2 (default) to add MNIST to the node
  * Confirm default tags by hitting "y" and ENTER
  * Pick the folder where MNIST is downloaded (this is due torch issue https://github.com/pytorch/vision/issues/3549)
  * Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
  
2. Check that your data has been added by executing `./scripts/fedbiomed_run node list`
3. Run the node using `./scripts/fedbiomed_run node run`. Wait until you get `Starting task manager`. it means you are online.

## Define an experiment model and parameters"

Declare a torch.nn MyTrainingPlan class to send for training on the node

In [2]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from torchvision import datasets, transforms

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(MyTrainingPlan, self).__init__(model_args)
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms"]
        
        self.add_dependency(deps)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        
        
        output = F.log_softmax(x, dim=1)
        return output

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        return DataManager(dataset=dataset1, **train_kwargs)
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss


This group of arguments correspond respectively:
* `model_args`: a dictionary with the arguments related to the model (e.g. number of layers, features, etc.). This will be passed to the model class on the node side.
* `training_args`: a dictionary containing the arguments for the training routine (e.g. batch size, learning rate, epochs, etc.). This will be passed to the routine on the node side.

**NOTE:** typos and/or lack of positional (required) arguments will raise error. 🤓

In [5]:
model_args = {}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

## Declare and run the experiment

- search nodes serving data for these `tags`, optionally filter on a list of node ID with `nodes`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `round_limit` rounds, applying the `node_selection_strategy` between the rounds

In [8]:
from fedbiomed.researcher.experiment_jl import ExperimentJL
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 10

exp = ExperimentJL(tags=tags,
                 model_args=model_args,
                 model_class=MyTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

2022-09-02 14:18:57,357 fedbiomed INFO - Searching dataset with data tags: ['#MNIST', '#dataset'] for all nodes
2022-09-02 14:19:07,369 fedbiomed INFO - Node selected for training -> node_b7843934-7037-4ec0-809f-41538850d760
2022-09-02 14:19:07,372 fedbiomed INFO - Node selected for training -> node_ba731923-352e-49eb-bd39-a95937842cdb
2022-09-02 14:19:07,376 fedbiomed INFO - Checking data quality of federated datasets...
2022-09-02 14:19:07,387 fedbiomed DEBUG - Model file has been saved: /workspaces/PycharmProjects/fedbiomed/var/experiments/Experiment_0028/my_model_4e01d3da-e862-4bbc-aa64-446e1230d7d1.py
2022-09-02 14:19:07,404 fedbiomed DEBUG - upload (HTTP POST request) of file /workspaces/PycharmProjects/fedbiomed/var/experiments/Experiment_0028/my_model_4e01d3da-e862-4bbc-aa64-446e1230d7d1.py successful, with status code 201
2022-09-02 14:19:07,504 fedbiomed DEBUG - upload (HTTP POST request) of file /workspaces/PycharmProjects/fedbiomed/var/experiments/Experiment_0028/aggregated

Let's start the experiment.

By default, this function doesn't stop until all the `round_limit` rounds are done for all the nodes

In [ ]:
exp.run(rounds=rounds, increase=True)

2022-09-02 14:20:17,326 fedbiomed INFO - Sampled nodes in round 0 ['node_b7843934-7037-4ec0-809f-41538850d760', 'node_ba731923-352e-49eb-bd39-a95937842cdb']
2022-09-02 14:20:17,327 fedbiomed INFO - Sending request 
					 To: node_b7843934-7037-4ec0-809f-41538850d760 
					 Request: : Perform training with the arguments: {'researcher_id': 'researcher_f07a5365-9039-453c-94c8-41b2ad3a2c8c', 'job_id': '1784fdb1-1680-4648-89de-075c92916d10', 'current_round': 0, 'training_args': scheme:
{'lr': {'rules': [<class 'float'>, <function TrainingArgs._lr_hook at 0x7f89a63e1700>], 'required': False}, 'batch_size': {'rules': [<class 'int'>], 'required': False}, 'epochs': {'rules': [<class 'int'>], 'required': False}, 'dry_run': {'rules': [<class 'bool'>], 'required': False}, 'batch_maxnum': {'rules': [<class 'int'>], 'required': False}, 'test_ratio': {'rules': [<class 'float'>, <function TrainingArgs._test_ratio_hook at 0x7f89a63e15e0>], 'required': False, 'default': 0.0}, 'test_on_local_updates': {'

2022-09-02 14:20:22,889 fedbiomed INFO - TRAINING 
					 NODE_ID: node_b7843934-7037-4ec0-809f-41538850d760 
					 Epoch: 1 | Completed: 3840/60000 (6%) 
 					 Loss: 0.286305 
					 ---------
2022-09-02 14:20:23,426 fedbiomed INFO - TRAINING 
					 NODE_ID: node_ba731923-352e-49eb-bd39-a95937842cdb 
					 Epoch: 1 | Completed: 4320/60000 (7%) 
 					 Loss: 0.425062 
					 ---------
2022-09-02 14:20:23,582 fedbiomed INFO - TRAINING 
					 NODE_ID: node_b7843934-7037-4ec0-809f-41538850d760 
					 Epoch: 1 | Completed: 4320/60000 (7%) 
 					 Loss: 0.292294 
					 ---------
2022-09-02 14:20:24,106 fedbiomed INFO - INFO
					 NODE node_ba731923-352e-49eb-bd39-a95937842cdb
					 MESSAGE: Reached 100 batches for this epoch, ignore remaining data
-----------------------------------------------------------------
2022-09-02 14:20:24,158 fedbiomed INFO - INFO
					 NODE node_ba731923-352e-49eb-bd39-a95937842cdb
					 MESSAGE: Quantization of model parameters
------------------------------------

2022-09-02 14:20:33,719 fedbiomed DEBUG - researcher_f07a5365-9039-453c-94c8-41b2ad3a2c8c
2022-09-02 14:20:33,758 fedbiomed INFO - WARNING
					 NODE node_b7843934-7037-4ec0-809f-41538850d760
					 MESSAGE: There is no validation activated for the round. Please set flag for `test_on_global_updates`, `test_on_local_updates`, or both. Splitting dataset for validation will be ignored
-----------------------------------------------------------------
2022-09-02 14:20:33,785 fedbiomed INFO - INFO
					 NODE node_b7843934-7037-4ec0-809f-41538850d760
					 MESSAGE: training with arguments {'history_monitor': <fedbiomed.node.history_monitor.HistoryMonitor object at 0x7f3630c63520>, 'node_args': {'gpu': False, 'gpu_num': None, 'gpu_only': False}, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}
-----------------------------------------------------------------
2022-09-02 14:20:33,826 fedbiomed INFO - WARNING
					 NODE node_ba731923-352e-49eb-bd39-a95937842cdb
					 MESSAGE: There

2022-09-02 14:20:42,703 fedbiomed INFO - INFO
					 NODE node_ba731923-352e-49eb-bd39-a95937842cdb
					 MESSAGE: results uploaded successfully 
-----------------------------------------------------------------
2022-09-02 14:20:48,736 fedbiomed INFO - Downloading model params after training on node_b7843934-7037-4ec0-809f-41538850d760 - from http://localhost:8844/media/uploads/2022/09/02/node_params_8b9dd7bb-25d4-4bc9-be4f-7bd46afa08bb.pkl
2022-09-02 14:20:48,755 fedbiomed DEBUG - upload (HTTP GET request) of file node_params_1d88d120-6f72-436d-a8ef-8d6a82efea16.pkl successful, with status code 200
2022-09-02 14:20:48,797 fedbiomed INFO - Model params downloaded
2022-09-02 14:20:48,798 fedbiomed INFO - Downloading model params after training on node_ba731923-352e-49eb-bd39-a95937842cdb - from http://localhost:8844/media/uploads/2022/09/02/node_params_04e40cb8-ed57-416e-8159-9f2c931fbc5d.pkl
2022-09-02 14:20:48,809 fedbiomed DEBUG - upload (HTTP GET request) of file node_params_ff4cb912

2022-09-02 14:20:52,906 fedbiomed INFO - TRAINING 
					 NODE_ID: node_b7843934-7037-4ec0-809f-41538850d760 
					 Epoch: 1 | Completed: 1920/60000 (3%) 
 					 Loss: 0.106755 
					 ---------
2022-09-02 14:20:52,979 fedbiomed INFO - TRAINING 
					 NODE_ID: node_ba731923-352e-49eb-bd39-a95937842cdb 
					 Epoch: 1 | Completed: 1920/60000 (3%) 
 					 Loss: 0.067903 
					 ---------
2022-09-02 14:20:53,665 fedbiomed INFO - TRAINING 
					 NODE_ID: node_b7843934-7037-4ec0-809f-41538850d760 
					 Epoch: 1 | Completed: 2400/60000 (4%) 
 					 Loss: 0.089584 
					 ---------
2022-09-02 14:20:53,694 fedbiomed INFO - TRAINING 
					 NODE_ID: node_ba731923-352e-49eb-bd39-a95937842cdb 
					 Epoch: 1 | Completed: 2400/60000 (4%) 
 					 Loss: 0.383159 
					 ---------
2022-09-02 14:20:54,368 fedbiomed INFO - TRAINING 
					 NODE_ID: node_ba731923-352e-49eb-bd39-a95937842cdb 
					 Epoch: 1 | Completed: 2880/60000 (5%) 
 					 Loss: 0.460734 
					 ---------
2022-09-02 14:20:54,433 fedbiomed I

2022-09-02 14:21:06,514 fedbiomed DEBUG - researcher_f07a5365-9039-453c-94c8-41b2ad3a2c8c
2022-09-02 14:21:06,515 fedbiomed INFO - Sending request 
					 To: node_ba731923-352e-49eb-bd39-a95937842cdb 
					 Request: : Perform training with the arguments: {'researcher_id': 'researcher_f07a5365-9039-453c-94c8-41b2ad3a2c8c', 'job_id': '1784fdb1-1680-4648-89de-075c92916d10', 'current_round': 3, 'training_args': {'batch_size': 48, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100, 'test_ratio': 0.0, 'test_on_local_updates': False, 'test_on_global_updates': False, 'test_metric': None, 'test_metric_args': {}}, 'training': True, 'model_args': {}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/09/02/my_model_4e01d3da-e862-4bbc-aa64-446e1230d7d1.py', 'params_url': 'http://localhost:8844/media/uploads/2022/09/02/aggregated_params_8b4a65d3-d22d-4984-ba5d-8eac826a7feb.pt', 'model_class': 'MyTrainingPlan', 'training_data': {'node_ba731923-352e-49eb-bd39-a959

2022-09-02 14:21:14,531 fedbiomed INFO - INFO
					 NODE node_ba731923-352e-49eb-bd39-a95937842cdb
					 MESSAGE: Quantization of model parameters done
-----------------------------------------------------------------
2022-09-02 14:21:14,536 fedbiomed INFO - INFO
					 NODE node_ba731923-352e-49eb-bd39-a95937842cdb
					 MESSAGE: Protecting model parameters with key <UserKey 0x0>
-----------------------------------------------------------------
2022-09-02 14:21:15,009 fedbiomed INFO - INFO
					 NODE node_b7843934-7037-4ec0-809f-41538850d760
					 MESSAGE: Protecting model parameters done
-----------------------------------------------------------------
2022-09-02 14:21:15,069 fedbiomed INFO - INFO
					 NODE node_ba731923-352e-49eb-bd39-a95937842cdb
					 MESSAGE: Protecting model parameters done
-----------------------------------------------------------------
2022-09-02 14:21:15,184 fedbiomed INFO - INFO
					 NODE node_b7843934-7037-4ec0-809f-41538850d760
					 MESSAGE: results upl

2022-09-02 14:21:23,715 fedbiomed INFO - TRAINING 
					 NODE_ID: node_ba731923-352e-49eb-bd39-a95937842cdb 
					 Epoch: 1 | Completed: 480/60000 (1%) 
 					 Loss: 0.096459 
					 ---------
2022-09-02 14:21:24,237 fedbiomed INFO - TRAINING 
					 NODE_ID: node_b7843934-7037-4ec0-809f-41538850d760 
					 Epoch: 1 | Completed: 960/60000 (2%) 
 					 Loss: 0.040413 
					 ---------
2022-09-02 14:21:24,387 fedbiomed INFO - TRAINING 
					 NODE_ID: node_ba731923-352e-49eb-bd39-a95937842cdb 
					 Epoch: 1 | Completed: 960/60000 (2%) 
 					 Loss: 0.084770 
					 ---------
2022-09-02 14:21:24,859 fedbiomed INFO - TRAINING 
					 NODE_ID: node_b7843934-7037-4ec0-809f-41538850d760 
					 Epoch: 1 | Completed: 1440/60000 (2%) 
 					 Loss: 0.048418 
					 ---------
2022-09-02 14:21:25,028 fedbiomed INFO - TRAINING 
					 NODE_ID: node_ba731923-352e-49eb-bd39-a95937842cdb 
					 Epoch: 1 | Completed: 1440/60000 (2%) 
 					 Loss: 0.086750 
					 ---------
2022-09-02 14:21:25,466 fedbiomed INFO

Local training results for each round and each node are available via `exp.training_replies()` (index 0 to (`rounds` - 1) ).

For example you can view the training results for the last round below.

Different timings (in seconds) are reported for each dataset of a node participating in a round :
- `rtime_training` real time (clock time) spent in the training function on the node
- `ptime_training` process time (user and system CPU) spent in the training function on the node
- `rtime_total` real time (clock time) spent in the researcher between sending the request and handling the response, at the `Job()` layer

In [ ]:
print("\nList the training rounds : ", exp.training_replies().keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies()[rounds - 1].data()
for c in range(len(round_data)):
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['node_id'],
        rtraining = round_data[c]['timing']['rtime_training'],
        ptraining = round_data[c]['timing']['ptime_training'],
        rtotal = round_data[c]['timing']['rtime_total']))
print('\n')
    
exp.training_replies()[rounds - 1].dataframe()

Federated parameters for each round are available via `exp.aggregated_params()` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [ ]:
print("\nList the training rounds : ", exp.aggregated_params().keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params()[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params()[rounds - 1]['params'].keys())


Feel free to run other sample notebooks or try your own models :D